In [1]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

class TestLogPosteriorAndGradient(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libminus_log_posterior_and_gradient.so')
        self.lib.minus_log_posterior_and_gradient.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
        ]
        self.lib.minus_log_posterior_and_gradient.restype = None

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: combined function correctly implemented')

    def test_minus_log_posterior_and_gradient(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        mlp_out = np.zeros((1,), dtype=np.float64)
        grad_out = np.zeros((47,), dtype=np.float64)

        # Call the Python method
        mpl_python, grad_python = self.model._minus_log_posteriorAndGradient(self.params)

        # Call the C++ function
        self.lib.minus_log_posterior_and_gradient(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau,
            mlp_out,
            grad_out
        )

        # Check if outputs match
        np.testing.assert_almost_equal(mlp_out[0], mpl_python, decimal=5, err_msg="Log-posterior mismatch")
        np.testing.assert_almost_equal(grad_out, grad_python, decimal=5, err_msg="Gradient mismatch")
# Run in a jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestLogPosteriorAndGradient)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

#if __name__ == '__main__':
#    unittest.main()

test_minus_log_posterior_and_gradient (__main__.TestLogPosteriorAndGradient.test_minus_log_posterior_and_gradient) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.953s

OK


Test passed: combined function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [5]:
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_and_gradient.so')

# Define argument and return types for the C++ function
lib.minus_log_posterior_and_gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.minus_log_posterior_and_gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Call the Python method
python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

# Prepare arrays to hold C++ results
cpp_loss = np.zeros((1,), dtype=np.float64)
cpp_grad = np.zeros((47,), dtype=np.float64)

# Call the C++ function
lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

# Print the results
print("Python loss:", python_loss)
print("Python gradient:", python_grad)
print("C++ loss:", cpp_loss[0])
print("C++ gradient:", cpp_grad)

Python loss: 506.85141876508357
Python gradient: [ 0.887452    1.2119592  20.81043235 20.77353514 20.73663447 20.69962126
 20.66310004 20.62675576 20.59051055 20.55434845 20.51881302 20.48361713
 20.44871255 20.41418152 20.38053984 20.3475265  20.31511807 20.28348768
 20.25303518 20.22358631 20.1951577  20.1680839  20.14266732 20.11876486
 20.09638126 20.07603215 20.05784719  0.15901247  0.15518859  0.14005269
  0.12646162  0.11193658  0.10796484  0.10490425  0.10839404  0.1156479
  0.11541665  0.06647619  0.10499837  0.12244192  0.12949048  0.12887341
  0.12187286  0.11522651  0.11226406  0.10990551  0.11285963]
C++ loss: 506.85141876508357
C++ gradient: [ 0.887452    1.2119592  20.81043235 20.77353514 20.73663447 20.69962126
 20.66310004 20.62675576 20.59051055 20.55434845 20.51881302 20.48361713
 20.44871255 20.41418152 20.38053984 20.3475265  20.31511807 20.28348768
 20.25303518 20.22358631 20.1951577  20.1680839  20.14266732 20.11876486
 20.09638126 20.07603215 20.05784719  0.1590

In [7]:
%%timeit -r 100 -n 100

lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

433 µs ± 76.8 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [8]:
%%timeit -r 100 -n 100

python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

1.43 ms ± 361 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
